In [ ]:
!git clone --single-branch --branch fast_tokenizers_BARTpho_PhoBERT_BERTweet https://github.com/datquocnguyen/transformers.git
%cd transformers
!pip3 install --upgrade .
%cd ..

fatal: destination path 'transformers' already exists and is not an empty directory.
/content/transformers
Processing /content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.32.0.dev0-py3-none-any.whl size=7416891 sha256=a8e6e67793cd1fd53c2f80ae0e7a7169178a25026842b8a03011c16bba5264b1
  Stored in directory: /tmp/pip-ephem-wheel-cache-h_sbwo6c/wheels/7c/35/80/e946b22a081210c6642e607ed65b2a5b9a4d9259695ee2caf5
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.32.0.dev0
    Uninstalling transformers-4.32.0.dev0:
      Successfully uninstalled transformers-4.32.0.dev0


/content


In [ ]:
!pip install torch torchvision transformers py_vncorenlp

In [ ]:
from transformers import AutoModel, AutoTokenizer

import torch

# Load pre-trained PhoBERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
model = AutoModel.from_pretrained("vinai/phobert-base")

# Example input text
text = "PhoBERT is a powerful language model for Vietnamese text."

# Tokenize and obtain embeddings
input_ids = tokenizer.encode(text, return_tensors="pt")
with torch.no_grad():
    embeddings = model(input_ids)["last_hidden_state"]

print("Embeddings shape:", embeddings.shape)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Embeddings shape: torch.Size([1, 19, 768])


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

# Load your dataset (replace 'your_dataset.csv' with your actual file)
comment_df = pd.read_csv('/content/drive/MyDrive/CS114/ABSA/rawdata.csv')
comment_df = comment_df[:224]
print(comment_df)
label_df = pd.read_csv('/content/drive/MyDrive/CS114/ABSA/label.csv', delimiter=';')

# Tokenize and encode your text data
encoded_data = tokenizer(comment_df['Comment'].tolist(), padding=True, truncation=True, return_tensors="pt")

# Convert labels to a multi-hot encoded format
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(label_df['label'].apply(lambda x: x.split(',')))

print(labels)
print(torch.sum(torch.tensor(labels), dim=1))

# Train-test split
# X_train, X_test, y_train, y_test = train_test_split(encoded_data['input_ids'], encoded_data['attention_mask'], labels, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
train_data = torch.utils.data.TensorDataset(encoded_data["input_ids"], encoded_data["attention_mask"], torch.tensor(labels, dtype=torch.float32))
test_data = torch.utils.data.TensorDataset(encoded_data["input_ids"], encoded_data["attention_mask"], torch.tensor(labels, dtype=torch.float32))

                                               Comment
0                     Dortmund bị psg khắc chế ảo thật
1    PSG phải chi có Xavi Simons chia lửa vs Mbapee...
2                        Kolo chuyền nguu mà sút cx gỗ
3    M3p chơi rất hay anh ta ko như một số tiền đạo...
4                                   Dót mun bán độ rồi
..                                                 ...
219  T tức cái thằng Barcola g đấy ghê á trời ơi là...
220                        Thời bóng đá của NGÁO VAR 😢
221  var quả này sai vc đập ức lên tay mà ai tính t...
222                                               Psg❤
223                         trận đấu đậm mùi dầu mỏ😂😂😂

[224 rows x 1 columns]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]]
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [ ]:
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm import tqdm

class SentimentAnalysisModel(nn.Module):
    def __init__(self, phobert_model, num_labels):
        super(SentimentAnalysisModel, self).__init__()
        self.phobert = phobert_model
        self.fc = nn.Linear(phobert_model.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.phobert(input_ids, attention_mask=attention_mask)
        logits = self.fc(outputs.last_hidden_state[:, 0, :])
        return torch.sigmoid(logits)

# Instantiate the model
num_labels = len(mlb.classes_)
model = SentimentAnalysisModel(model, num_labels)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# DataLoader for training and testing
batch_size = 8
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Training loop
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    training_loss = 0.0
    for batch in tqdm(train_loader):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        training_loss += loss.item()
        loss.backward()
        optimizer.step()
    avg_training_loss = training_loss / len(train_loader)
    # Validation
    model.eval()
    with torch.no_grad():
        total_loss = 0.0
        for batch in test_loader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            outputs = model(input_ids, attention_mask)
            total_loss += criterion(outputs, labels).item()

    avg_loss = total_loss / len(test_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Trainging Loss: {avg_training_loss}, Validation Loss: {avg_loss}")

# Save the trained model
torch.save(model.state_dict(), "aspect_sentiment_model.pth")

100%|██████████| 28/28 [00:05<00:00,  4.67it/s]


Epoch 1/10, Trainging Loss: 0.8693635570151466, Validation Loss: 0.8136746010610035


100%|██████████| 28/28 [00:06<00:00,  4.48it/s]


Epoch 2/10, Trainging Loss: 0.7989735241447177, Validation Loss: 0.7751776639904294


100%|██████████| 28/28 [00:06<00:00,  4.53it/s]


...

Epoch 8/10, Trainging Loss: 0.7432046553918293, Validation Loss: 0.7370552676064628


100%|██████████| 28/28 [00:05<00:00,  4.80it/s]


Epoch 9/10, Trainging Loss: 0.7405859551259449, Validation Loss: 0.7347624791519982


100%|██████████| 28/28 [00:05<00:00,  4.82it/s]


Epoch 10/10, Trainging Loss: 0.737957530788013, Validation Loss: 0.7326619050332478


In [ ]:
sentence = ['benzema hay qua']
encoded_data = tokenizer(sentence, padding=True, truncation=True, return_tensors="pt").to(device)
outputs = model(encoded_data['input_ids'], encoded_data['attention_mask'])

In [ ]:
outputs

tensor([[0.0941, 0.0879, 0.1027, 0.0859, 0.0838, 0.0827, 0.1216, 0.0991, 0.0917,
         0.0819, 0.1006, 0.1194]], device='cuda:0', grad_fn=<SigmoidBackward0>)

In [ ]:
import numpy as np
prob = outputs.cpu().detach().numpy()
mapped_array = np.where(prob > 0.11, 1, 0)
original_labels = mlb.inverse_transform(mapped_array)
original_labels

[('Neutral#other', 'Positive#player')]